# TX00DQ05-3001 Exercises 3

In [2]:
import numpy as np
import numpy.linalg as LA
import random

## Exercise 1: Sample behaviour of an MDP

Let's take (again) a look at Sutton & Barto example 4.1 gridworld. On each iteration start at every (non-terminating) state and sample actions in succeeding states by selecting them from uniform distribution (each action - up, down, left, right - is equally probable). Run the episode until terminal state is encountered. Collect statistics to calculate average number of steps needed before completion for each start state. Should this number match with something you have seen earlier in the exercises?

In [61]:
STATES = []

def calculateValueForCurrentState(value,stepcost):
    #print("(",stepcost,"+",value,")*0.25 = ",((stepcost+(value))*0.25))
    return ((stepcost+(value))*0.25)


def checkWithBoundary(r,c,oR,oC):
    #print("-nstate: ",r,",",c)
    #print(V)
    if 0 > r or 4 <= r or 0 > c or 4 <= c :
        #print(oR,",",oC," value:",V[oR,oC])
        return [oR,oC]
    else:
        #print(r,",",c," value:",V[r,c])
        return [r,c]

    
    
def Action_Policy(val):
    if val == 0:
        return [1,0]
    elif val == 1:
        return [-1,0]
    elif val == 2:
        return [0,1]
    elif val == 3:
        return [0,-1]
    

def randomAction():
    return Action_Policy(random.randint(0,3))

def makeAction(v,r,c):
    a = randomAction()
    
    
    return checkWithBoundary(r+a[0],c+a[1],r,c)

def checkStates(r,c):
    for s in STATES:
        if (r,c) in s:
            return True
    STATES.append([r,c])    
    return False
    
    
def MC_pathlengths(maxiters,term,v,r,c):
    lengths = dict()
    # YOUR CODE
    leng = []
    l=0
    row = r
    col = c
    new_state = [0,0]
    for i in range(maxiters):
        new_state = makeAction(v,row,col)
        row = new_state[0]
        col = new_state[1]
        if checkStates(row,col) == False:
            if (row,col) in term:
                leng.append(l)
            
                l = 0
            else:
                l+=1
            
    #print(leng)
    return leng

def calculatePathMean(lengths):
    return np.mean(lengths)
        
    
    
def IterateValue(gridworld,terminating,stepcost,eps,maxiters):
    
    V = gridworld
    V_new = np.zeros((V.shape[0], V.shape[1]))
    #for i in range(maxiters):
    # YOUR CODE HERE
    #print(V)
    lenght =0
    for c in range(V.shape[0]):
        for r in range(V.shape[1]):
            #print("in (",r,",",c,")")
            if not (r, c) in terminating:
                lenght=calculatePathMean(MC_pathlengths(maxiters,terminating,V,r,c))
                #V_new[r,c] = calculateValueForCurrentState(lenght,stepcost)
                V_new[r,c] = lenght
        
    print(V_new)
    return V_new


IterateValue(V_new,terminating,stepcost,eps,maxiters)   

[[ 0.          6.15827338  6.14705882  9.01020408]
 [ 7.37815126  6.43181818  6.34814815  7.12195122]
 [10.2         5.26582278  8.30188679  7.        ]
 [ 4.58100559  8.37735849  4.57541899  0.        ]]


array([[ 0.        ,  6.15827338,  6.14705882,  9.01020408],
       [ 7.37815126,  6.43181818,  6.34814815,  7.12195122],
       [10.2       ,  5.26582278,  8.30188679,  7.        ],
       [ 4.58100559,  8.37735849,  4.57541899,  0.        ]])

In [23]:
gridSize = (4,4)
actions = [
    (0,-1), #up
    (0, 1), #down
    (-1, 0),#left
    (1, 0)  #right
]
terminationPoints = [(0,0), (3,3)]

def terminationPoint(pos):
    for t in terminationPoints:
        if pos == t:
            return True
    return False

def doAction(pos):
    a = actions[random.randint(0,3)]
    newPos = tuple(map(lambda x, y: x + y, pos, a))
    # Check boundaries
    if (newPos[0] >= gridSize[0]):
        newPos = (gridSize[0]-1, pos[1])
    if (newPos[0] < 0):
        newPos = (0, pos[1])
    if (newPos[1] >= gridSize[1]):
        newPos = (pos[0], gridSize[1]-1)
    if (newPos[1] < 0):
        newPos = (pos[0], 0)
    
    return newPos

class Agent(object):
    steps = 0
    def __init__(self, startPos):
        self.startPos = startPos
        self.pos = startPos

def MC_pathlengths(maxiters):
    lengths = np.zeros(gridSize)
    
    # Initialize agents
    agentList = []
    aIndex = 0
    for x in range(gridSize[0]):
        for y in range(gridSize[1]):
            if not terminationPoint((x,y)):
                agentList.append( Agent((x,y)) )
            aIndex += 1
    for i in range(1, maxiters):
        # List for agents who have reached termination point
        stoppedAgentList = []
        
        # Loop while there is still agents who have not
        # reached term point
        while (len(agentList) > 0):
            for agent in agentList:
                agent.pos = doAction(agent.pos)
                agent.steps += 1
                if (terminationPoint(agent.pos)):
                    agentList.remove(agent)
                    stoppedAgentList.append(agent)
        
        # Calculate new mean and reset agents
        for agent in stoppedAgentList:
            mean = ((i-1)/i) * lengths[agent.startPos] + agent.steps/i
            print(((i-1)/i),"*",lengths[agent.startPos],"+",agent.steps/i,"=",mean)
            lengths[agent.startPos] = mean
            agent.steps = 0
            agent.pos = agent.startPos
            
        #Put back for next iteration
        agentList = stoppedAgentList
            
    return lengths

print(MC_pathlengths(10))

0.0 * 0.0 + 1.0 = 1.0
0.0 * 0.0 + 2.0 = 2.0
0.0 * 0.0 + 8.0 = 8.0
0.0 * 0.0 + 7.0 = 7.0
0.0 * 0.0 + 9.0 = 9.0
0.0 * 0.0 + 15.0 = 15.0
0.0 * 0.0 + 17.0 = 17.0
0.0 * 0.0 + 25.0 = 25.0
0.0 * 0.0 + 30.0 = 30.0
0.0 * 0.0 + 30.0 = 30.0
0.0 * 0.0 + 34.0 = 34.0
0.0 * 0.0 + 39.0 = 39.0
0.0 * 0.0 + 44.0 = 44.0
0.0 * 0.0 + 59.0 = 59.0
0.5 * 1.0 + 0.5 = 1.0
0.5 * 7.0 + 0.5 = 4.0
0.5 * 44.0 + 1.0 = 23.0
0.5 * 30.0 + 2.5 = 17.5
0.5 * 59.0 + 2.5 = 32.0
0.5 * 8.0 + 3.5 = 7.5
0.5 * 30.0 + 6.0 = 21.0
0.5 * 9.0 + 7.5 = 12.0
0.5 * 2.0 + 9.0 = 10.0
0.5 * 25.0 + 15.0 = 27.5
0.5 * 17.0 + 16.5 = 25.0
0.5 * 39.0 + 21.0 = 40.5
0.5 * 34.0 + 20.0 = 37.0
0.5 * 15.0 + 29.0 = 36.5
0.6666666666666666 * 7.5 + 0.3333333333333333 = 5.333333333333333
0.6666666666666666 * 4.0 + 0.6666666666666666 = 3.333333333333333
0.6666666666666666 * 10.0 + 0.6666666666666666 = 7.333333333333333
0.6666666666666666 * 27.5 + 0.6666666666666666 = 19.0
0.6666666666666666 * 25.0 + 1.6666666666666667 = 18.333333333333332
0.6666666666666666 *

In [49]:
# YOUR CODE

def MC_pathlengths(maxiters):
    lengths = dict()
    # YOUR CODE

    return lengths


## Exercise 2: Monte Carlo state value function estimation. 

Calculate state-value function V for the gridworld of Sutton & Barto example 4.1 using first-visit or every-visit Monte Carlo policy evaluation (see for example page 92 of Sutton & Barto). Policy to be evaluated is the same as before; each action (up, down, left, right) is equally probable.  Action that would result in leaving the grid (for example moving up in top row) will leave state unchanged (but action has been taken). Gamma (discount factor) is assumed to be = 1, ie. no discounting.

Try out both exploring starts (see Sutton & Barto, p. 96) and fixed start points. Any difference?

Take a look at the value function you get when you run the algorithm multiple times (with fixed # of iterations). Any observations?

[[  0.   -14.89 -20.72 -22.48]
 [-14.33 -18.32 -20.87 -21.31]
 [-20.25 -20.02 -18.3  -14.76]
 [-23.01 -20.63 -14.43   0.  ]]


## Exercise 3*: Monte Carlo action value function estimation

Use the same idea as in exercise 2 to estimate q function.

*) - not mandatory

## Exercise 4*: Monte Carlo control

Compute the optimal policy for the 4x4 gridworld example. Start with random policy. Consider the epsilon adjustment schedule - can it in practise be 1/k, or is something more conservative better? Can you think of any other tricks to manage the noisiness of MC?

*) - not mandatory